In [30]:
import cv2
import mediapipe as mp
import numpy as np
import os


In [31]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,  
    min_detection_confidence=0.8, 
    min_tracking_confidence=0.8
)
mp_drawing = mp.solutions.drawing_utils

In [32]:
GESTURE = "Handshake"
SAVE_DIR = f"data/{GESTURE}"
os.makedirs(SAVE_DIR, exist_ok=True)

cap = cv2.VideoCapture(0)
count = 0

def extract_keypoints(results):
    keypoints = np.zeros(126) 

    if results.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            for j, landmark in enumerate(hand_landmarks.landmark):
                index = (i * 63) + (j * 3)  
                keypoints[index:index + 3] = [landmark.x, landmark.y, landmark.z]

    return keypoints


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image)


    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        
        keypoints = extract_keypoints(results)
        np.save(f"{SAVE_DIR}/{count}.npy", keypoints)
        count += 1

    
    cv2.putText(frame, f"Recording {GESTURE}: {count}/500", (10, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Recording Sign Gestures (Both Hands)', frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q') or count >= 500:
        break

In [33]:
cap.release()
cv2.destroyAllWindows()

print(f" Data collection for {GESTURE} completed successfully!")

 Data collection for Handshake completed successfully!
